# Deep Q-Network (DQN)
---
In this notebook, you will implement a DQN agent with OpenAI Gym's LunarLander-v2 environment.

### 1. Import the Necessary Packages

In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import tqdm as tqdm
from butifarra_env import ButifarraEnv
from collections import defaultdict
%matplotlib inline

### 2. Instantiate the Environment and Agent

Initialize the environment in the code cell below.

In [2]:
env = ButifarraEnv(render_mode='ansi')
env.reset()
print('State shape: ', env.observation_space(env.agents[0])["observation"].shape)
print('Number of actions: ', env.action_space(env.agents[0]).n)

State shape:  (628,)
Number of actions:  48


Before running the next code cell, familiarize yourself with the code in **Step 2** and **Step 3** of this notebook, along with the code in `dqn_agent.py` and `model.py`.  Once you have an understanding of how the different files work together, 
- Define a neural network architecture in `model.py` that maps states to action values.  This file is mostly empty - it's up to you to define your own deep Q-network!
- Finish the `learn` method in the `Agent` class in `dqn_agent.py`.  The sampled batch of experience tuples is already provided for you; you need only use the local and target Q-networks to compute the loss, before taking a step towards minimizing the loss.

Once you have completed the code in `dqn_agent.py` and `model.py`, run the code cell below.  (_If you end up needing to make multiple changes and get unexpected behavior, please restart the kernel and run the cells from the beginning of the notebook!_)

You can find the solution files, along with saved model weights for a trained agent, in the `solution/` folder.  (_Note that there are many ways to solve this exercise, and the "solution" is just one way of approaching the problem, to yield a trained agent._)

### 3. Train the Agent with DQN

Run the code cell below to train the agent from scratch.  You are welcome to amend the supplied values of the parameters in the function, to try to see if you can get better performance!

In [3]:
%tensorboard --logdir=runs


UsageError: Line magic function `%tensorboard` not found.


In [5]:
from dqn_agent import Agent, RandomAgent
env.reset()
agents = {agent_name: Agent(state_size=env.observation_space(env.agents[0])["observation"].shape[0], action_size=env.action_space(env.agents[0]).n, seed=0, suffix=agent_name)
          for agent_name in env.agents}
# agents['player_0'] = RandomAgent(suffix='random_0')
# agents['player_2'] = RandomAgent(suffix='random_2')

def dqn(n_episodes=3000, max_t=500, eps_start=1.0, eps_end=0.01, eps_decay=0.9995):
    """Deep Q-Learning.

    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = defaultdict(list)                        # list containing scores from each episode
    scores_window = defaultdict(lambda: deque(maxlen=100))  # last 100 scores
    eps = eps_start                    # initialize epsilon
    best_score = -np.inf
    mean_score = defaultdict(float)
    bar = tqdm.tqdm(range(1, n_episodes+1))
    for i_episode in bar:
        env.reset()
        observations = {}
        score = defaultdict(float)
        for agent_name in env.agent_iter():
            next_state, reward, d1, d2, _ = env.last()
            if d1 or d2:
                action = None
            else:
                action = agents[agent_name].act(next_state, eps, env.action_space(agent_name))
            env.step(action)
            score[agent_name] += reward
            done = d1 or d2
            if agent_name in observations:
                agents[agent_name].step(observations[agent_name], action, reward, next_state["observation"].astype(np.float32), done)
            observations[agent_name] = next_state["observation"].astype(np.float32)
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        for agent_name in agents.keys():
            agents[agent_name].report_score(score[agent_name])
            scores_window[agent_name].append(score[agent_name])       # save most recent score
            scores[agent_name].append(score[agent_name])              # save most recent score
            mean_score[agent_name] = np.mean(scores_window[agent_name])
        bar.set_postfix(eps=eps, **mean_score)
        # if i_episode % 100 == 0:
        #     if mean_score > best_score:
        #         best_score = mean_score
        #         # torch.save(agent.qnetwork_local.state_dict(), f'checkpoint_buti.pth')

    return scores


dqn()

  0%|          | 0/3000 [00:00<?, ?it/s]

 81%|████████  | 2425/3000 [09:26<02:14,  4.28it/s, eps=0.297, player_0=-13.4, player_1=-28.5, player_2=-57.8, player_3=-28.3]


KeyboardInterrupt: 

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [10]:
# Run the trained agent
from butifarra_env import get_action

env.reset()
score=0
for agent_name in env.agent_iter():
    state, reward, d1, d2, _  = env.last()
    print(env.render())
    if d1 or d2:
        action = None
    else:
        if agent_name == 'player_0':
            score += reward
            print(f'Score: {score}')
            print(f"Mask : {state['action_mask']}")
            while True:
                try:
                    card = input('Enter action: ')
                    if card == 'pass':
                        action = None
                        break
                    action = get_action(card)
                    break
                except:
                    print('Invalid card')
        else:
            agents[agent_name].use_mask = True
            action = agents[agent_name].act(state, 0.0, env.action_space(agent_name))

    env.step(action)

print(f"Score: {score}")


Agent player_0 has hand [6O, 7O, 10O, 9O, 6C, 7C, 10C, 6E, 11E, AE, 2B, 9B]
Agent player_1 has hand [12O, 2C, 5C, 8C, 11C, 9C, 12E, 3B, 5B, 6B, 7B, 11B]
Agent player_2 has hand [2O, 3O, 4O, 3C, 4C, 12C, 2E, 4E, 8E, 4B, 8B, AB]
Agent player_3 has hand [5O, 8O, 11O, AO, AC, 3E, 5E, 7E, 10E, 9E, 10B, 12B]
Trump is E
Table is []
Last trick won by 02 team is []
Last trick won by 13 team is []

Score: 0
Mask : [0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1
 0 0 0 0 0 0 0 0 0 0 1]
Invalid card
Agent player_0 has hand [6O, 7O, 10O, 9O, 6C, 7C, 6E, 11E, AE, 2B, 9B]
Agent player_1 has hand [12O, 2C, 5C, 8C, 11C, 9C, 12E, 3B, 5B, 6B, 7B, 11B]
Agent player_2 has hand [2O, 3O, 4O, 3C, 4C, 12C, 2E, 4E, 8E, 4B, 8B, AB]
Agent player_3 has hand [5O, 8O, 11O, AO, AC, 3E, 5E, 7E, 10E, 9E, 10B, 12B]
Trump is E
Table is [10C]
Last trick won by 02 team is []
Last trick won by 13 team is []

Agent player_0 has hand [6O, 7O, 10O, 9O, 6C, 7C, 6E, 11E, AE, 2B, 9B]
Agent player_1 has 

AssertionError: action is not in action space

### 5. Explore

In this exercise, you have implemented a DQN agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task with discrete actions!
- You may like to implement some improvements such as prioritized experience replay, Double DQN, or Dueling DQN! 
- Write a blog post explaining the intuition behind the DQN algorithm and demonstrating how to use it to solve an RL environment of your choosing.  